In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import gc
import gzip
import re
import shutil
import glob
import multiprocessing as mp
import errno
from Bio import SeqIO

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import torch
from torch import nn
GPU_PRESENT = [torch.cuda.device (i) for i in range (torch.cuda.device_count ())]!=[]
print("GPU Detected?: "+str(GPU_PRESENT))
if GPU_PRESENT:
    torch.set_default_device('cuda')
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
    #import cupy as cp


fastqs = glob.glob('Data/fastq/*.fastq')

PAD_SIZE=1000
BATCH_SIZE=10000

GPU Detected?: True
Using cuda device


In [2]:
print(enc.fit_transform(np.array(["A","T", "C", "G"]).reshape(-1,1)).toarray())

def get_seqios(file):
    seqs = []
    with open(file) as fastq:
        for index, record in enumerate(SeqIO.parse(fastq, 'fastq')):
            seqs.append(str(record.seq))

    return seqs

def parse_reads(record, pad_size=PAD_SIZE):
    x_in = np.array(list(record))
    arr = enc.fit_transform(x_in.reshape(-1,1)).toarray()
    delta = len(arr)-pad_size

    if delta>0:
        #random crop
        shift=np.random.randint(0,delta)
        x_out = arr[shift:shift+pad_size]

    else:
        arr.resize((pad_size, 4), refcheck=False)
        x_out=arr

    return x_out

#Go get em!
seqios = []
for n in fastqs:
    reads=get_seqios(n)
    seqios+=reads

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


In [1]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, seqs):
        self.data = seqs

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        out = parse_reads(self.data[idx])
        out = out.reshape(PAD_SIZE, 4)
        out = torch.tensor(out).to(torch.float)
        return out
    
dataset = Dataset(seqios)

encoder = torch.nn.Sequential(
            torch.nn.Conv1d(INPUT_SHAPE, 400, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(400, 128, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(128, 64, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(64, 16, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(16, 9, 1)
        )

decoder = torch.nn.Sequential(
            torch.nn.Conv1d(9, 16, 4, 4),
            torch.nn.ReLU(),
            torch.nn.Conv1d(16, 64, 4, 4),
            torch.nn.ReLU(),
            torch.nn.Conv1d(64, 128, 4 , 4),
            torch.nn.ReLU(),
            torch.nn.Conv1d(128, 400, 4, 4),
            torch.nn.ReLU(),
            torch.nn.Conv1d(400, INPUT_SHAPE, 1, 1),
            torch.nn.Sigmoid()
        )

#you can only get values out by "iterating"

    
att = torch.nn.MultiheadAttention(4,4,  dropout=0.05)

c = encoder(b)
l = torch.nn.Conv1d(9, 16, 1)
l(c)

NameError: name 'torch' is not defined

In [ ]:
INPUT_SHAPE=PAD_SIZE
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv1d(INPUT_SHAPE, 400, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(400, 128, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(128, 64, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(64, 16, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(16, 16, 1)
        )
         
        self.decoder = torch.nn.Sequential(
            torch.nn.Conv1d(16, 16, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(16, 64, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(64, 128, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(128, 400, 1),
            torch.nn.ReLU(),
            torch.nn.Conv1d(400, INPUT_SHAPE, 1, 1),
            torch.nn.Sigmoid()
        )
 
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Model Initialization
model = AE()

# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()

# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-5,weight_decay = 1e-4)
optimizer.zero_grad()

In [93]:
optimizer = torch.optim.Adam(model.parameters(),lr = 5e-5,weight_decay = 1e-9)

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=100,
                                         shuffle=True,
                                         num_workers=0,
                                         generator=torch.Generator(device='cuda'))
steps_per_epoch=200
num_epochs=10
losses=[]
for epoch in range(num_epochs):
    for batch_index, doc in enumerate(dataloader):
        recon = model(doc)
        #Loss function
        loss = loss_function(recon, doc)
        if batch_index%10==0:
            print("Batch: "+str(batch_index))
            print("loss"+str(loss))

        # Gradients are set to zero,
        # Gradient is computed and stored.
        # .step() performs parameter update.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Storing the losses
        losses.append(loss)

Batch: 0
losstensor(0.1247, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 10
losstensor(0.1320, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 20
losstensor(0.1141, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 30
losstensor(0.1153, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 40
losstensor(0.1129, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 50
losstensor(0.1175, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 60
losstensor(0.1243, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 70
losstensor(0.1167, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 80
losstensor(0.1160, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 90
losstensor(0.1143, device='cuda:0', grad_fn=<MseLossBackward0>)
Batch: 100
losstensor(0.1171, device='cuda:0', grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 

In [97]:
torch.save(model.state_dict(), "./RNA_Autoencoder.state_dict")

'''
model = AE()
model.load_state_dict(torch.load(./RNA_Autoencoder.state_dict))
model.eval()
'''

In [99]:
model.decoder(model.encoder(b))

tensor([[2.1789e-01, 2.4013e-01, 2.6422e-01, 2.5691e-01],
        [2.7289e-01, 2.5482e-01, 2.3385e-01, 2.3641e-01],
        [2.8707e-01, 2.6344e-01, 2.2969e-01, 2.2858e-01],
        ...,
        [2.6640e-07, 3.0567e-07, 3.9485e-07, 4.0044e-07],
        [1.8912e-07, 2.1102e-07, 2.7574e-07, 2.8464e-07],
        [3.6691e-07, 4.1822e-07, 5.3359e-07, 5.3885e-07]], device='cuda:0',
       grad_fn=<SigmoidBackward0>)

In [74]:
batch_index

0